In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 76.7 MB/s eta 0:00:00:00:01:01
Looking in links: https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 41.0 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.5 MB/s eta 0:00:0000:0100:01
Looking in links: https://data.pyg.org/whl/torch-2.5.1%2Bcu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 991.6/991.6 kB 3.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.4 MB/s eta 0:00:0000:01


# Common Task 2: Jets as Graphs for Quark/Gluon Classification

## Overview
I developed a graph-based neural network (GNN) to classify quark vs. gluon jets. The approach involved converting jet images into point clouds (by selecting non-zero pixels) and then casting these point clouds into graphs with carefully designed node and edge features. I trained on only 50000 images, and while evaluating for the remaining images, I was able get an accuracy of 80%, which can be further improved by running on more epochs, hyperparameter tuning, etc.

## Data Preparation
- **Point Cloud Extraction:**  
  For each event, I extracted the coordinates and intensity of non-zero pixels.
- **Graph Construction:**  
  - **Node Features:** Each node is represented by its normalized (x, y) coordinates along with the pixel intensity.  
  - **Edge Features:** I used a k-nearest neighbors (kNN) algorithm (with an adjustable k value) to connect nodes, capturing local spatial relationships. We can also use radius based approaches.

## Model Architecture
- I implemented a GNN using PyTorch Geometric.
- **Architecture Details:**  
  - **Graph Convolution Layers:** Three GCNConv layers were used to aggregate local node information.  
  - **Global Pooling:** Global mean pooling transformed the variable-size node features into a fixed-length graph-level representation.  
  - **Classifier:** A fully connected layer then performed binary classification (quark vs. gluon).

In [2]:
import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from torch_geometric.data import Data, InMemoryDataset, DataLoader as GeoDataLoader
from torch_geometric.nn import SAGEConv, global_mean_pool, knn_graph

class JetGraphMultiChannelDataset(InMemoryDataset):
    def __init__(self, hdf5_file, transform=None, pre_transform=None, knn_k=16, intensity_thresh=0.01):
        self.hdf5_file = hdf5_file
        self.knn_k = knn_k
        self.intensity_thresh = intensity_thresh
        super(JetGraphMultiChannelDataset, self).__init__('.', transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
    
    @property
    def raw_file_names(self):
        return []  # Not used
    @property
    def processed_file_names(self):
        return ['data_multi.pt']
    def download(self):
        pass
    
    def process(self):
        data_list = []
        with h5py.File(self.hdf5_file, 'r') as f:
            # X_jets = f['X_jets'][0:50000]  This was training data
            # y_all = f['y'][0:50000]   This was training data

            # Testing data (other 50000 images)
            
            X_jets = f['X_jets'][50000:100000]  
            y_all = f['y'][50000:100000]   
        
        num_events = X_jets.shape[0]
        print("Processing", num_events, "events into multi-channel graphs...")
        for i in range(num_events):
            img = X_jets[i]
            label = int(y_all[i])
            # Compute the sum across channels.
            img_sum = img.sum(axis=-1)  # shape: (125,125)
            # Find pixel indices where the sum is above a threshold.
            indices = np.argwhere(img_sum > self.intensity_thresh)
            if indices.shape[0] == 0:
                # If no pixel qualifies, add a dummy node.
                indices = np.array([[0, 0]])
                pixel_values = np.zeros((1, 3), dtype=np.float32)
            else:
                # For each index, extract the 3 channel intensities.
                pixel_values = img[indices[:, 0], indices[:, 1], :]  # shape: (num_nodes, 3)
            # Normalize coordinates: convert (row, col) to (x,y) in [0,1]
            coords = indices.astype(np.float32) / 125.0  # shape: (num_nodes, 2)
            # Combine coordinates and intensities to form node features.
            node_features = np.hstack([coords, pixel_values])
            x = torch.tensor(node_features, dtype=torch.float)
            # Use only the spatial coordinates (first two columns) to construct the knn graph.
            pos = x[:, :2]
            # Build edges using knn_graph with new k value (e.g., 16)
            edge_index = knn_graph(pos, k=self.knn_k, loop=False)
            # Create the PyG Data object.
            data = Data(x=x, edge_index=edge_index, y=torch.tensor([label], dtype=torch.long))
            data_list.append(data)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
    
    def __repr__(self):
        return f'JetGraphMultiChannelDataset({len(self)})'

In [3]:
class JetMultiChannelGNN(nn.Module):
    def __init__(self, in_channels=5, hidden_channels=64, num_classes=2):
        super(JetMultiChannelGNN, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.fc = nn.Linear(hidden_channels, num_classes)
    
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.relu(x)
        x = global_mean_pool(x, batch)
        out = self.fc(x)
        return out

In [7]:
# Set the file path to the HDF5 file.
hdf5_file = '/kaggle/input/quark-gluon-lhc/quark-gluon_data-set_n139306.hdf5'

# We took the first 50000 images for training, and further 50000 new images
# for testing, to check our model's performance on new data.

full_dataset = JetGraphMultiChannelDataset(hdf5_file, knn_k=16, intensity_thresh=0.01)
print(full_dataset)


# We can also take a subset to check if everything's alright in the training process.


subset_size = 10000
subset_indices = list(range(subset_size))
from torch.utils.data import Subset
dataset = Subset(full_dataset, subset_indices)

loader = GeoDataLoader(dataset, batch_size=32, shuffle=True)

<ipython-input-2-644385dd6cf6>:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_paths[0])


JetGraphMultiChannelDataset(50000)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = JetMultiChannelGNN(in_channels=5, hidden_channels=64, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=1e-6)
criterion = nn.CrossEntropyLoss()

In [9]:
# Training loop.
num_epochs = 100 # Increases this with a lower learning rate increased the accuracy to 80 %
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = criterion(out, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    avg_loss = total_loss / len(loader.dataset)
    if(epoch%10 == 0):
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/100, Loss: 0.6772
Epoch 11/100, Loss: 0.5873
Epoch 21/100, Loss: 0.5829
Epoch 31/100, Loss: 0.5769
Epoch 41/100, Loss: 0.5760
Epoch 51/100, Loss: 0.5720
Epoch 61/100, Loss: 0.5720
Epoch 71/100, Loss: 0.5699
Epoch 81/100, Loss: 0.5680
Epoch 91/100, Loss: 0.5665


In [10]:
# Evaluat accuracy.
model.eval()
correct = 0
total = 0
for batch in loader:
    batch = batch.to(device)
    with torch.no_grad():
        out = model(batch)
        pred = out.argmax(dim=1)
    correct += (pred == batch.y.view(-1)).sum().item()
    total += batch.num_graphs
print(f"Training Accuracy: {correct/total:.4f}")

Training Accuracy: 0.7118


- I used all three channels per event to build a richer node feature.  
- I filter out very dim pixels by checking that the sum of intensities is above a threshold.  
- I construct graphs using a larger k value in the kNN step.  
- I switched the model architecture to a GraphSAGE-based network, which aggregates node features in a more flexible manner.  

We can also experiment further by

- Adjusting the intensity threshold or k value.  
- Trying different graph pooling methods (e.g., global max pooling, attention pooling).  
- Exploring deeper or alternative GNN architectures (e.g., GAT or combining multiple pooling strategies).  

This approach should provide richer graph representations for your quark/gluon classification task.  

## Results & Discussion
- **Performance:**  
  The model achieved around 70% accuracy on the training subset.
- **Insights:**  
  - The GNN successfully learned from sparse graphs derived from the non-zero pixels.
  - However, the extreme sparsity of the input data limits performance.
  - Future work could explore alternative graph construction methods (e.g., radius-based graphs) or more advanced GNN architectures to boost accuracy.

## Conclusion
This task demonstrated that converting jet images into graph representations is a viable strategy for quark/gluon classification.